In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn import linear_model

In [2]:
np.arange(0,10).reshape(1,-1)

df = pd.DataFrame()
df['x'] = np.arange(0,10)
df['y'] = np.arange(0,10)
df['z'] = np.arange(0,10)

reg = linear_model.LinearRegression()
reg.fit(df[['x','y']], df['z'])
print(reg.coef_)
print(reg.feature_names_in_)

# for path in Path('./Models/Models').rglob('*.csv'):
#     print(path.name)
    
    

[0.5 0.5]
['x' 'y']
